<a href="https://colab.research.google.com/github/melissatorgbi/LLMCxG_Workshop/blob/main/notebooks/LLMCxG_Notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

overview/description

## Imports

In [1]:
import csv
import pandas as pd
import random
from openai import OpenAI
import re
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Set-up

In [2]:
OPENAI_API_KEY = #enter your api key here

In [3]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
def get_response(prompt, model):

  completion = client.chat.completions.create(
      model=model,
      messages=[
          {
              "role": "user",
              "content": prompt
          }
      ]
  )

  response = completion.choices[0].message.content
  return response

In [5]:
def clean_sentences(sentences) :
    clean_sentences = list()
    for sentence in sentences :
        sentence = re.sub( r"\s*\([^\)]*?\)\s*$", r"", sentence )
        # sentence = re.sub( r"\s+.$", '.', sentence )
        if sentence != '' :
            new_sentence = list()
            for word, pos in nltk.pos_tag( sentence.split( ' ' ) ) :
                verbs   = [ (i.pos()=='v') for i in wordnet.synsets(word) ]
                wn_verb = verbs.count( True )  > verbs.count( False )
                if word.isupper() and ( 'VB' in pos or wn_verb ) :
                    word = word.lower()
                new_sentence.append( word )
            sentence = ' '.join( new_sentence )
            clean_sentences.append( sentence )
    return clean_sentences

In [6]:
def clean_data(df):
  for column in df.columns:
    clean_list = clean_sentences(df[column])
    df[column] = clean_list

  return df

## Load Data

In [7]:
!git clone https://github.com/H-TayyarMadabushi/Construction_Grammar_Schematicity_Corpus-CoGS.git

fatal: destination path 'Construction_Grammar_Schematicity_Corpus-CoGS' already exists and is not an empty directory.


In [8]:
cogs_df = pd.read_csv("Construction_Grammar_Schematicity_Corpus-CoGS/Dataset/CoGs.csv")
cogs_df = cogs_df.loc[:, ~cogs_df.columns.str.match('Unnamed')]
cogs_df = cogs_df.loc[0:49]
cogs_df.head()

Let Alone  \
0  Most wives are too bloody old, let alone mothe...   
1  A ceasefire, let alone lasting peace, will tak...   
2  It is difficult enough for an individual to be...   
3  I would be distressed to hear of any ladies re...   
4  None of these arguments is notably strong, let...   

                                          Way Manner  \
0  Tricia backed her way out, never taking her ey...   
1  The taxi nosed its way back into the traffic a...   
2  As she felt her way forward, suddenly a knight...   
3  If this were to happen, would it not be unfair...   
4  Through the French windows and across the lawn...   

                                         Resultative  \
0    The man shrieked himself unconscious. (Hoffman)   
1           Firefighters cut the man free. (Hoffman)   
2        He had often drank himself silly. (Hoffman)   
3  It JERKS you awake with the first sentence , h...   
4  These agencies RENDER themselves ineffective b...   

                                            Conative  \
0                  She kicked at the ball. (Hoffman)   
1               He clutched at the branch. (Hoffman)   
2                They shot at the sheriff. (Hoffman)   
3  With her free hand, she tugged at the strap of...   
4              She tugged at the glass door.  (COCA)   

                                 Intransitive Motion  \
0            The fly buzzed into the room. (Hoffman)   
1                 He ran out of the house. (Hoffman)   
2         People strolled along the river. (Hoffman)   
3  It shouldn't be - you should be able to DANCE ...   
4  When the IED detonates, this copper cup turns ...   

                                       Caused Motion  \
0           He sang them out of the room.  (Hoffman)   
1   She wiggled her feet out of the boots. (Hoffman)   
2    They laughed the actor off the stage. (Hoffman)   
3          I THREW the stone across the river. (AMR)   
4  The RN can still launch a task force that woul...   

                                  Causative with CxN  \
0         She loaded the truck with books. (Hoffman)   
1         He sprayed the walls with paint. (Hoffman)   
2  They heaped the plate with mashed potatoes. (H...   
3  Kevin lifted the trumpet and filled the room w...   
4  Dude has filled the room with flowers and tedd...   

                                    Ditransitive CxN  \
0                Jack passed her the salt. (Hoffman)   
1     The waiter served them their dinner. (Hoffman)   
2                   She sent him an email. (Hoffman)   
3  The above kind of exchange AFFORDS students th...   
4            She baked her sister a cake. (Goldberg)   

                            Comparative Correlative   \
0  The harder they come, the harder they fall. (AMR)   
1  The longer he is around, the more miserable I ...   
2   The more I studied, the less I understood. (AMR)   
3                        The more the merrier. (AMR)   
4  It seems the older the patient, the less effec...   

                                          Much Less   
0  When my dad catches swarms sometimes he doesn'...  
1  Not that many of us can fly off to Guatemala, ...  
2  I never thought I'd meet him, much less in Aus...  
3  Those were not my first (much less only) thoug...  
4  You can not hear (much less understand) anythi...

In [9]:
cogs_df["Let Alone"][0]

'Most wives are too bloody old, let alone mothers. (FN Construction)'

# Clean Data

In [10]:
cogs_df = clean_data(cogs_df)

In [11]:
cogs_df["Let Alone"][0]

'Most wives are too bloody old, let alone mothers.'

# Constructing Prompts

In [12]:
construction = "Let Alone"
construction_sentence = cogs_df[construction][0]
remaining_constructions = cogs_df.drop(columns = construction)
random_construction1 = random.choice(remaining_constructions.columns)
alternative_sentence1 = cogs_df[random_construction1][0]
random_construction2 = random.choice(remaining_constructions.columns)
remaining_constructions = remaining_constructions.drop(columns = random_construction1)
alternative_sentence2 = cogs_df[random_construction2][0]

print("Target Construction: {}\nSentence: {}\n".format(construction, construction_sentence))
print("Alternative Construction: {}\nSentence: {}\n".format(random_construction1, alternative_sentence1))
print("Alternative Construction: {}\nSentence: {}\n".format(random_construction2, alternative_sentence2))

Target Construction: Let Alone
Sentence: Most wives are too bloody old, let alone mothers.

Alternative Construction: Much Less 
Sentence: When my dad catches swarms sometimes he doesn't even wear a veil, much less a bee suit.

Alternative Construction: Caused Motion
Sentence: He sang them out of the room.



In [13]:
prompt_base = """Question: Which of the following sentence are instances of the {} construction? Output nothing but the relevant sentence:
{}
{}
{}
Answer:
"""

In [14]:
prompt = prompt_base.format(construction, construction_sentence, alternative_sentence1, alternative_sentence2)
print(prompt)

Question: Which of the following sentence are instances of the Let Alone construction? Output nothing but the relevant sentence: 
Most wives are too bloody old, let alone mothers.
When my dad catches swarms sometimes he doesn't even wear a veil, much less a bee suit.
He sang them out of the room.
Answer:



# Prompting the Model

In [15]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

response = completion.choices[0].message.content


In [16]:
print(response)

Most wives are too bloody old, let alone mothers.  
When my dad catches swarms sometimes he doesn't even wear a veil, much less a bee suit.


# Evaluation

In [17]:
print(construction_sentence)

Most wives are too bloody old, let alone mothers.


In [18]:
print("scores_exact", response == construction_sentence )
print("scores_contained", construction_sentence.lower() in response.lower() )
print("scores_contains", response.lower() in construction_sentence.lower() )

scores_exact False
scores_contained True
scores_contains False


# Prompting with Multiple Examples

In [19]:
def get_rows(df, row_num):

  rows = []

  for i in range(row_num):
    row = []
    construction = random.choice(df.columns)
    construction_sentence = df[construction][random.randint(0, df[construction].count()-2)]

    remaining_constructions = df.drop(columns = construction)
    random_construction1 = random.choice(remaining_constructions.columns)
    alternative_sentence1 = df[random_construction1][random.randint(0, df[random_construction1].count()-2)]

    random_construction2 = random.choice(remaining_constructions.columns)
    remaining_constructions = remaining_constructions.drop(columns = random_construction1)
    alternative_sentence2 = df[random_construction2][random.randint(0, df[random_construction2].count()-2)]

    row = [construction_sentence, alternative_sentence1, alternative_sentence2]
    random.shuffle(row)
    row.append(construction)
    row.append(construction_sentence)

    rows.append(row)

  new_df = pd.DataFrame(rows, columns = ['sentence1','sentence2','sentence3','construction','target'])

  return new_df


In [20]:
cogsn_df = get_rows(cogs_df, 10)

In [21]:
cogsn_df.head(10)

sentence1  \
0  The squire looked at the peasant and covered h...   
1  Justin Timberlake makes Alcoholism Fun In The ...   
2  Most stable-lads would have counted themselves...   
3  So the more we talk about it, the more coverag...   
4  These practical experiences afford school psyc...   
5                            I ran around the track.   
6  Marc seems to this there should be more news r...   
7  Her fingers plucked at the air in movements as...   
8  He did not sound a bit sorry and she almost sm...   
9  The idea of everyone writing a letter for thei...   

                                           sentence2  \
0                 The joggers ran the pavement thin.   
1  I thought about taking Phoneix's advice and tu...   
2  No one mentioned abortion, much less birth con...   
3  The district court judge in Colorado awarded t...   
4  He had just danced a circle around the Grim Re...   
5  He does NOT belong in the USA, much less the o...   
6       He stabbed at the last part of the equation.   
7  Workers dumped large burlap sacks of the impor...   
8  The more private sources run charities, the le...   
9  China confirmed on January 23, 2007 after 12 d...   

                                           sentence3  \
0  He buys her a bag of anise-flavored candies fr...   
1  This because Anons are not looking for persona...   
2                   The sleeping pills made me sick.   
3           Martin dabbed her lips with poppy paste.   
4  Dude has filled the room with flowers and tedd...   
5  But it happened that after walking for a long ...   
6  James Jackson and Company of Augusta , the sec...   
7  Bill, the more I read your stuff, the more I a...   
8  we did not see a single boat on our three dive...   
9  The Pakistan-born man was then deported to the...   

               construction                                             target  
0               Resultative                 The joggers ran the pavement thin.  
1               Resultative  Justin Timberlake makes Alcoholism Fun In The ...  
2               Resultative                   The sleeping pills made me sick.  
3        Causative with CxN           Martin dabbed her lips with poppy paste.  
4          Ditransitive CxN  These practical experiences afford school psyc...  
5                Much Less   He does NOT belong in the USA, much less the o...  
6          Ditransitive CxN  James Jackson and Company of Augusta , the sec...  
7  Comparative Correlative   Bill, the more I read your stuff, the more I a...  
8                Way Manner  He did not sound a bit sorry and she almost sm...  
9        Causative with CxN  The idea of everyone writing a letter for thei...

In [22]:
exact_scores = []
relaxed_scores = []
responses = []

for index in range(len(cogsn_df)):
  prompt = prompt_base.format(
      cogsn_df.construction.iloc[index],
      cogsn_df.sentence1.iloc[index],
      cogsn_df.sentence2.iloc[index],
      cogsn_df.sentence3.iloc[index])

  target = cogsn_df.target.iloc[index]
  response = get_response(prompt, "gpt-4o-mini")

  exact_score = response == target
  relaxed_score = target.lower() in response.lower() or response.lower() in target.lower()

  responses.append(response)
  exact_scores.append(exact_score)
  relaxed_scores.append(relaxed_score)

cogsn_df["response"] = responses
cogsn_df["exact score"] = exact_scores
cogsn_df["relaxed score"] = exact_scores

In [23]:
cogsn_df.head(10)

sentence1  \
0  The squire looked at the peasant and covered h...   
1  Justin Timberlake makes Alcoholism Fun In The ...   
2  Most stable-lads would have counted themselves...   
3  So the more we talk about it, the more coverag...   
4  These practical experiences afford school psyc...   
5                            I ran around the track.   
6  Marc seems to this there should be more news r...   
7  Her fingers plucked at the air in movements as...   
8  He did not sound a bit sorry and she almost sm...   
9  The idea of everyone writing a letter for thei...   

                                           sentence2  \
0                 The joggers ran the pavement thin.   
1  I thought about taking Phoneix's advice and tu...   
2  No one mentioned abortion, much less birth con...   
3  The district court judge in Colorado awarded t...   
4  He had just danced a circle around the Grim Re...   
5  He does NOT belong in the USA, much less the o...   
6       He stabbed at the last part of the equation.   
7  Workers dumped large burlap sacks of the impor...   
8  The more private sources run charities, the le...   
9  China confirmed on January 23, 2007 after 12 d...   

                                           sentence3  \
0  He buys her a bag of anise-flavored candies fr...   
1  This because Anons are not looking for persona...   
2                   The sleeping pills made me sick.   
3           Martin dabbed her lips with poppy paste.   
4  Dude has filled the room with flowers and tedd...   
5  But it happened that after walking for a long ...   
6  James Jackson and Company of Augusta , the sec...   
7  Bill, the more I read your stuff, the more I a...   
8  we did not see a single boat on our three dive...   
9  The Pakistan-born man was then deported to the...   

               construction  \
0               Resultative   
1               Resultative   
2               Resultative   
3        Causative with CxN   
4          Ditransitive CxN   
5                Much Less    
6          Ditransitive CxN   
7  Comparative Correlative    
8                Way Manner   
9        Causative with CxN   

                                              target  \
0                 The joggers ran the pavement thin.   
1  Justin Timberlake makes Alcoholism Fun In The ...   
2                   The sleeping pills made me sick.   
3           Martin dabbed her lips with poppy paste.   
4  These practical experiences afford school psyc...   
5  He does NOT belong in the USA, much less the o...   
6  James Jackson and Company of Augusta , the sec...   
7  Bill, the more I read your stuff, the more I a...   
8  He did not sound a bit sorry and she almost sm...   
9  The idea of everyone writing a letter for thei...   

                                            response  exact score  \
0                 The joggers ran the pavement thin.         True   
1  None of the sentences provided are instances o...        False   
2                   The sleeping pills made me sick.         True   
3  The district court judge in Colorado awarded t...        False   
4  These practical experiences afford school psyc...        False   
5  He does NOT belong in the USA, much less the o...         True   
6  James Jackson and Company of Augusta , the sec...         True   
7  Bill, the more I read your stuff, the more I a...         True   
8  He did not sound a bit sorry and she almost sm...         True   
9  None of the sentences provided are instances o...        False   

   relaxed score  
0           True  
1          False  
2           True  
3          False  
4          False  
5           True  
6           True  
7           True  
8           True  
9          False

In [24]:
exact_accuracy = sum(cogsn_df["exact score"])/len(cogsn_df) * 100
relaxed_accuracy = sum(cogsn_df["relaxed score"])/len(cogsn_df) * 100

print(exact_accuracy)
print(relaxed_accuracy)

60.0
60.0


# Test Data

In [25]:
test_df = pd.read_csv("Construction_Grammar_Schematicity_Corpus-CoGS/Dataset/TestData.csv")

In [26]:
test_df.head()

Construction Name                                    Test Sentence 1  \
0         Let Alone  It described extravagant Easter egg packaging ...   
1         Let Alone  so many kids are born in sterile hospital room...   
2         Let Alone  How many modern Prime Ministers could recall s...   
3         Let Alone  Largely preoccupied by a sense of not being ab...   
4         Let Alone  It is difficult enough for an individual to be...   

                                     Test Sentence 2  \
0  A ceasefire, let alone lasting peace, will tak...   
1             Let me alone; my days have no meaning.   
2                The old man would not let it alone.   
3  Promises of a new bullet train here, a relief ...   
4  I would be distressed to hear of any ladies re...   

                                     Test Sentence 3  \
0                      I'll let you alone right now.   
1  You'd have trouble swinging a gerbil, let alon...   
2  It takes science to come along and tell us tha...   
3       Try to be happy ... let the glass globe be .   
4                      I'll let you alone right now.   

                                     Test Sentence 4  \
0       Try to be happy ... let the glass globe be .   
1  It takes science to come along and tell us tha...   
2                      I'll let you alone right now.   
3  so many kids are born in sterile hospital room...   
4  It's unsurprising that such an attitude failed...   

                                     Test Sentence 5  \
0                       They can't let nature alone.   
1       Try to be happy ... let the glass globe be .   
2  It is difficult enough for an individual to be...   
3  How many modern Prime Ministers could recall s...   
4  so many kids are born in sterile hospital room...   

                                     Test Sentence 6  \
0  Promises of a new bullet train here, a relief ...   
1  The Protestant majority in Ulster totally refu...   
2                       They can't let nature alone.   
3                The old man would not let it alone.   
4                The old man would not let it alone.   

   Test Correct Sentence 1 Index  Test Correct Sentence 2 Index  \
0                              0                              1   
1                              2                              3   
2                              0                              2   
3                              0                              1   
4                              0                              1   

   Test Correct Sentence 3 Index  ...  \
0                              5  ...   
1                              5  ...   
2                              4  ...   
3                              4  ...   
4                              3  ...   

                                 One-shot Sentence 1  \
0  The first membranes they made were far too thi...   
1                The old man would not let it alone.   
2             Let me alone; my days have no meaning.   
3  The director-general, Michael Checkland, once ...   
4  The first membranes they made were far too thi...   

                                 One-shot Sentence 2  \
0             Let me alone; my days have no meaning.   
1                      I'll let you alone right now.   
2  so many kids are born in sterile hospital room...   
3                       They can't let nature alone.   
4       Try to be happy ... let the glass globe be .   

                                 One-shot Sentence 3  \
0  The director-general, Michael Checkland, once ...   
1  Where Musgrove and John Hopkins, who put it al...   
2       Try to be happy ... let the glass globe be .   
3                      I'll let you alone right now.   
4             Let me alone; my days have no meaning.   

                                  One-shotSentence 4  \
0  However, its role in influencing these rhythms...   
1  The first membranes they made were far too thi...   
2  The disadvantage was that 'the liberal b